In [1]:
import sys


from platform import python_version

print(python_version())

3.13.1


`id()` è una funzione built-in in Python. Ci dà la possibilità di controllare **l'identificatore univoco** di un oggetto. In CPython, esso coincide con l'indirizzo di memoria RAM in cui si trova l'oggetto. Diamo un'occhiata a come funziona. Ricordiamo che in Python **tutto è un oggetto**, visto che a basso livello tutto punta ad una struttura dati complessa.

Ad esempio, questa è la struttura utilizzata nell'implementazione di Python più diffusa (CPython, cioè Python basato su C) per un `long integer`:

    struct _longobject {
        long ob_refcnt;
        PyTypeObject *ob_type;
        size_t ob_size;
        long ob_digit[1];
    };
    
`ob_refcnt`, a reference count that helps Python silently handle memory allocation and deallocation<br />
`ob_type`, which encodes the type of the variable<br />
`ob_size`, which specifies the size of the following data members<br />
`ob_digit`, which contains the actual integer value that we expect the Python variable to represent.

Assegnamo ad `a` il valore `500` e controlliamone l'id.

In [2]:
a = 500

print(a)
print(id(a))
print(hex(id(a)))

500
4464604304
0x10a1c7490


`id()` punta a una posizione specifica in memoria. Facciamo qualche test:

In [3]:
a = 500
b = 500

print(a == b)
print(id(a) == id(b))
print(a is b)

True
False
False


Ecco la prima "stranezza". Abbiamo due comportamenti diversi a seconda dell'operatore che stiamo utilizzando:
- `==` confronta i **valori** degli operandi
- `is` confronta gli **id** degli operandi

#### N.B. alla luce di ciò, confrontare con un `==` due `id` di due variabili corrisponde ad usare l'operatore `is` fra le variabili

Vediamo cosa succede nel caso di stringhe:


In [4]:
a = "Test String"
b = "Test String"
print(a is b)
print(a == b)

False
True


Stessa cosa che abbiamo visto nel caso precedente.

In [5]:
list1 = [1, 2, 3]
list2 = list1

print(list1 is list2)
print(list1 == list2)

True
True


In questo caso abbiamo creato una lista e poi abbiamo assegnato ad un'altra variabile la lista appena creata. In casi come questi, si parla di `aliasing`. In realtà quello che abbiamo fatto è fare in modo che un **reference** della prima lista fosse copiato nella seconda lista. Questo ha delle implicazioni. Se facciamo una modifica ad un valore nella variabile `list2`, **il cambiamento si rifletterà anche sul `list1`**!



In [6]:
list2.append(4)
print("list1", list1)
print("list2", list2)

print(list1 is list2)
print(list1 == list2)

list1 [1, 2, 3, 4]
list2 [1, 2, 3, 4]
True
True


Ovviamente anche cambiare un valore non va a modificare l'id della lista: l'indirizzo di memoria rimarrà sempre quello anche dopo un'assegnazione del genere

In [7]:
my_list = ['Gatto', 'Cane', 'Coniglio']
print(my_list, '->', hex(id(my_list)))
my_list[0] = 'Mucca'
print(my_list, '->', hex(id(my_list)))

['Gatto', 'Cane', 'Coniglio'] -> 0x10a3b0bc0
['Mucca', 'Cane', 'Coniglio'] -> 0x10a3b0bc0


Il cambio dell'elemento nella lista è stato effettuato correttamente, ma l'indirizzo della variabile `my_list` **non è cambiato**. Il valore è stato cambiato sulla stessa copia della variabile originale. Questo è un esempio di **mutability**.

Vediamo che succede ora ai singoli valori della lista `my_list`:

In [8]:
print(f"Indirizzo della lista: {hex(id(my_list))}")
print(f"Indirizzo del primo el. della lista: {hex(id(my_list[0]))}")
my_list[0] = "Asino"
print(f"Indirizzo della lista: {hex(id(my_list))}")
print(f"Indirizzo del primo el. della lista: {hex(id(my_list[0]))}")

Indirizzo della lista: 0x10a3b0bc0
Indirizzo del primo el. della lista: 0x10a3e9200
Indirizzo della lista: 0x10a3b0bc0
Indirizzo del primo el. della lista: 0x10a3e9770


L'id del primo elemento è una certa locazione di memoria quando il primo elemento è "Mucca". Quando cambiamo il primo elemento e lo settiamo ad "Asino" **la cella di memoria in cui viene memorizzata la stringa cambia**. Invece la lista mantiene lo stesso indirizzo.

Un oggetto immutabile è un oggetto che **non è modificabile 'inline'** e il suo stato non può essere modificato dopo la sua creazione.
Ad esempio, una stringa è immutabile. Non è possibile sovrascrivere i valori di oggetti immutabili.

Tuttavia, è possibile assegnare nuovamente la variabile. Quindi non modifichiamo l'oggetto stringa, **ne creiamo uno ex-novo**.


In [9]:
s = "Oh i like you!"
print(hex(id(s)))

s = "Oh i hate you!"

print(hex(id(s)))

0x10a3e6eb0
0x10a3e6230


Dal momento che una stringa è immutabile, è stato creato un nuovo oggetto che punta ad un'altra locazione di memoria. 
Proviamo ad effettuare una modifica inline:

In [10]:
s[0] = "S"

TypeError: 'str' object does not support item assignment

Dal momento che le stringhe sono immutabili, Python ci restituisce un `TypeError`.


**Numeri, stringhe e tuple are immutabili**.<br>   
**Liste, dizionari, e set sono mutabili**, come la stragrande maggioranza degli oggetti creati dalle classi.

L'immutabilità può essere usata per assicurarsi che un oggetto rimanga "costante" durante l'esecuzione di un programma. I valori degli oggetti mutable possono invece essere cambiati "inline" in ogni momento. 

### Come sono passati gli argomenti ad una funzione e quali sono le conseguenze per oggetti mutabili e immutabili?

Il modo in cui il compilatore Python gestisce gli argomenti delle funzioni è strettamente correlato al fatto che essi siano mutabili o no.

Normalmente, <ins>un oggetto mutabile viene passato ad una funzione **per riferimento**</ins>. Questo significa che la variabile originale può essere modificata. Se si desidera evitare questo comportamento, è necessario copiare la variabile in un'altra variabile. Al contrario, <ins>un oggetto immutabile viene passato ad una funzione **per valore**</ins>. Quindi non è possibile modificare intrinsecamente il valore di quell'immutabile. Ecco alcuni esempi:

In [11]:
def inc(n):
    n += 1

a = 5
inc(a)
print(a)

5


La variabile `a` si riferisce all'oggetto con valore `5`. Quando lo passiamo alla funzione `increment`, la variabile locale `n` si riferisce allo stesso oggetto. Ad ogni modo, dal momento che `a` è un numero, e i numeri sono immutable, l'incremento di `n` avverrà solo all'interno della funzione. Quindi la stampa del valore di `a` dopo la chiamata a funzione restituirà sempre il valore originario: `5`. 

In [12]:
def inc(n):
    n['age'] = 37

d = {"first_name": "Giuseppe", "last_name": "Mastrandrea"}
inc(d)
print(d)

{'first_name': 'Giuseppe', 'last_name': 'Mastrandrea', 'age': 37}


Stavolta, siccome la variabile `l` contiene un oggetto *mutable*, ovvero un dizionario, la variabile locale `n` nella funzione `inc` punterà alla stessa cella di memoria a cui punta la variabile `d`. Quindi la modifica fatta in `inc` si rifletterà **anche** sulla variabile `d`. Nessun nuovo oggetto è creato. Viene modificato "inline" l'oggetto originario.



In [13]:
def copy_ref(n, v):
    n = v

list1 = [1, 2, 3]
list2 = [4, 5, 6]
copy_ref(list1, list2)
print(list1)

[1, 2, 3]


In questo caso passiamo alla funzione due liste. La funzione ha le varibili locali `n` e `v`, che si riferiscono rispettivamente allo stesso oggetto a cui si riferiscono `list1` e `list2`.

La funzione assegna `n` all'oggetto a cui si riferisce `v`. Quindi `n` e `v` si riferiscono allo stesso oggetto. 
Quindi `n`, `v` e `list2` si riferiscono tutte e tre a `[4, 5, 6]`. `list1`, al contrario, punta ancora all'oggetto `[1, 2, 3]`, che in effetti viene poi stampato nella `print` finale.

Come facciamo a copiare una lista? Ci sono molti modi, eccone uno piuttosto _naive_:

In [14]:
def copy_list(n):
    out = n[:]
    return out
    
l = [1, 2, 3]
nl = copy_list(l)

Passiamo a `copy_list` una lista per reference. La funzione conterrà una variabile locale (`n`) che punta allo stesso oggetto di `l`. Tuttavia, in questo caso, abbiamo usato lo slice operator (`:`) per creare una copia della lista e ritornarla in output alla funzione. In output avremo un reference a quella copia! Adesso `nl` si riferirà ad un oggetto diverso da quello originariamente passato alla funzione.


In [15]:
list_1 = [1, 2, 3]

list_2 = copy_list(list_1)

print(list_1 == list_2)
print(list_1 is list_2)
print('list_1', hex(id(list1)))
print('list_2', hex(id(list2)))

True
False
list_1 0x10bb3b0c0
list_2 0x10bb39f80


## Peculiarità

### Small Integers Caching

In [16]:
a = 1
b = 2
c = 2

print(hex(id(a)))
print(hex(id(b)))
print(hex(id(c)))

a = 250
b = 250

print("250 -> ", id(a)==id(b))

a = 260
b = 260
print("260 -> ", id(a)==id(b))

0x1046faa18
0x1046faa38
0x1046faa38
250 ->  True
260 ->  False


Notiamo che nonostante b e c siano due variabili diverse, esse hanno lo stesso id(); usando CPython, possiamo dire che queste variabili puntano alla **stessa cella di memoria**. Hanno lo stesso identificatore univoco perché `c` sta facendo riferimento a un oggetto che contiene il valore 2 e `b` sta anche facendo riferimento allo stesso oggetto che contiene il valore 2. Entrambi puntano allo stesso oggetto che contiene un valore 2. L'oggetto, `0x104e00110`, è l'identificatore univoco.

Perchè abbiamo un True per 250 e False per 260?
La ragione è che Python, per i cosiddetti "Small Integers" (ovvero quelli che vanno da -5 a 256) conserva un array dedicato in memoria. Quando creiamo un numero che si trova in quel range, otteniamo un reference ad un oggetto che **esiste già**. Questo è implementato in CPython grazie a delle macro, `NSMALLNEGINTS`, `NSMALLPOSINTS` e `CHECK_SMALL_INT`. 

    #ifndef NSMALLPOSINTS
    #define NSMALLPOSINTS           257
    #endif
    #ifndef NSMALLNEGINTS
    #define NSMALLNEGINTS           5
    #endif
    #define CHECK_SMALL_INT(ival)
        do if (-NSMALLNEGINTS <= ival && ival < NSMALLPOSINTS) {
            return get_small_int((sdigit)ival);
        } while(0)`

### Interning



In [17]:
a = "strawberry"
b = "strawberry"
print(id(a) == id(b))
print(a is b)
print(a == b)


True
True
True


L'`interning` è il processo di ottimizzazione della memoria per cui in presenza di stringhe uguali, esse puntano allo stesso oggetto in memoria. In sostanza, quindi, quando creiamo due stringhe con lo stesso valore, invece di allocare memoria per entrambe, Python memorizza *solo un* valore in memoria. L'altra stringa punta semplicemente alla stessa locazione di memoria della prima:


In [18]:
a = 'hello'
b = 'hello'
c = 'hell'

L'interning in questo caso avviene per a e b, mentre c è diversa e quindi non viene ri-utilizzata la locazione di memoria già allocata per a e b. Proviamolo:

In [19]:
print((hex(id(a))))
print((hex(id(b))))
print((hex(id(c))))

0x10960b300
0x10960b300
0x10a377900


Ora che abbiamo dimostrato che per c non c'è stato interning, possiamo controllare che l'uguaglianza del valore di c (

In [20]:
c = c + 'o'


print(a, c)
print(a is b)
print(a == b)

print(a is c)
print(a == c)

hello hello
True
True
False
True


È possibile effettuare un intern esplicito usando `sys.intern()`:

In [21]:
letter = 'd'

a = sys.intern('Hello World')
b = sys.intern('Hello Worl' + letter)

print(f"The ID of a: {hex(id(a))}")
print(f"The ID of b: {hex(id(b))}")
print(f"a is b? {a is b}")

The ID of a: 0x10a3c5070
The ID of b: 0x10a3c5070
a is b? True


### Tuple

L'immutabilità sulle tuple è vera solo in parte. La tupla in sè non può essere modificata, ma in alcuni casi **è possibile modificare gli oggetti in essa contenuti**. Se la tupla ha un campo immutabile come una stringa, allora la tupla non può essere modificata ed è talvolta chiamata "immutabilità non transitiva". Ma un campo mutabile come un elenco può essere modificato, come dimostrato dal seguente esempio:

In [22]:
t = (1, 2, ['Hello', 'World'])

print(t)

t[2].append('!!!')

print(t)

(1, 2, ['Hello', 'World'])
(1, 2, ['Hello', 'World', '!!!'])
